In [1]:
import rospy
import numpy as np
from panda_robot_custom import PandaRobotCustom
from tf.transformations import quaternion_from_euler, euler_from_quaternion
import my_log
import tools

log = my_log.logger()
log._print_logger_info()

rospy.init_node("franka_test", anonymous=True)

robot = PandaRobotCustom()
rospy.sleep(1) 

2023-08-20 10:59:32 [INFO] Logger 'my_log' with level 20
2023-08-20 10:59:32 [INFO] Initializing PandaRobot class
2023-08-20 10:59:33 [INFO] Setting stiffness
2023-08-20 10:59:33 [INFO] Old stiffness: {'translational_stiffness': 400.0, 'rotational_stiffness': 20.0, 'nullspace_stiffness': 1.0}
2023-08-20 10:59:35 [INFO] New stiffness: {'translational_stiffness': 400.0, 'rotational_stiffness': 20.0, 'nullspace_stiffness': 1.0}


/opt/ros/noetic/lib/python3/dist-packages/tf/listener.py:97: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(self._buffer.all_frames_as_yaml()) or {}


2023-08-20 10:59:38 [INFO] cart pose:
trans:[0.6006924076637438, 1.649841886362173e-05, 0.5817979188786024], rot: (3.0543638841508716, -0.1337138829758345, 0.053901348200239535)
2023-08-20 10:59:38 [INFO] q: (0.005007212772753178, -0.63794968641559, -0.04088317875048286, -2.216238004095228, -0.10126612873676066, 1.7185788964280828, 0.7156847328263556)
2023-08-20 10:59:41 [INFO] Panda initialization DONE


In [2]:
trans = [0.6, 0.0, 0.5]
rot = [np.pi, -np.pi/3, 0.0]
robot.cart_move(trans, quaternion_from_euler(rot[0], rot[1], rot[2]))

In [3]:
log.info(robot.get_cart_pose())

2023-08-20 10:59:54 [INFO] ([0.6001835129129361, 3.276035944555465e-06, 0.5810567667985753], [-0.9967380259734402, -0.026247301065226582, -0.06243418845256441, -0.043890305052832895])
